# Panle Regression - Firm Characteristics

### Random Effects Panel Regression

In [56]:
import pandas as pd
import numpy as np
import datetime as dt
import sklearn
import statsmodels.api as sm
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from linearmodels.panel import PanelOLS
from linearmodels.panel import RandomEffects


Data

In [57]:
df = pd.read_csv("Dataframes/macro_regression.csv")
df["Date"] = pd.to_datetime(df["Date"])
df

,Instrument,Date,GICS Industry Group Name,Earnings Per Share - Actual Surprise,Earnings Per Share - Actual Surprise AbsVals,CBOE Crude Oil ETF Volatility Index,90-Day AA Financial Commercial Paper Interest Rate,Inflation Risk Premium,"University of Michigan: Consumer Sentiment, Index 1966:Q1=100",Unemployment Rate
0,AVY.N,2013-01-01,Materials,11.178,11.178,22.538500,0.166000,0.359537,76.666667,7.733333
1,AVY.N,2013-01-04,Materials,2.482,2.482,23.468125,0.143750,0.360572,81.666667,7.533333
2,AVY.N,2013-01-07,Materials,1.068,1.068,24.099219,0.125625,0.463789,81.566667,7.233333
3,AVY.N,2013-01-10,Materials,8.095,8.095,19.740938,0.125806,0.441854,76.933333,6.933333
4,AVY.N,2014-01-01,Materials,1.471,1.471,19.563770,0.126230,0.465460,80.933333,6.666667
...,...,...,...,...,...,...,...,...,...,...
19196,POOL.OQ,2021-01-10,Retailing,17.194,17.194,43.036308,0.140755,0.382645,69.900000,4.200000
19197,POOL.OQ,2022-01-01,Retailing,40.267,40.267,52.695806,0.467234,0.380946,63.133333,3.800000
19198,POOL.OQ,2022-01-04,Retailing,34.342,34.342,50.457581,1.394118,0.495588,57.866667,3.600000
19199,POOL.OQ,2022-01-07,Retailing,1.503,1.503,49.861562,2.885882,0.423045,56.100000,3.566667


Standardising and log transformation

In [58]:
rescale = df.copy()
rescale["Earnings Per Share - Actual Surprise AbsVals"] = StandardScaler().fit_transform(np.array(rescale["Earnings Per Share - Actual Surprise AbsVals"]).reshape(-1,1))
rescale["CBOE Crude Oil ETF Volatility Index"] = StandardScaler().fit_transform(np.array(rescale["CBOE Crude Oil ETF Volatility Index"]).reshape(-1,1))
rescale["90-Day AA Financial Commercial Paper Interest Rate"] = StandardScaler().fit_transform(np.array(rescale["90-Day AA Financial Commercial Paper Interest Rate"]).reshape(-1,1))
rescale["Inflation Risk Premium"] = StandardScaler().fit_transform(np.array(rescale["Inflation Risk Premium"]).reshape(-1,1))
rescale["University of Michigan: Consumer Sentiment, Index 1966:Q1=100"] = StandardScaler().fit_transform(np.array(rescale["University of Michigan: Consumer Sentiment, Index 1966:Q1=100"]).reshape(-1,1))
rescale["Unemployment Rate"] = StandardScaler().fit_transform(np.array(rescale["Unemployment Rate"]).reshape(-1,1))

#log transform DV
rescale["Earnings Per Share - Actual Surprise AbsVals"] = np.log(rescale["Earnings Per Share - Actual Surprise AbsVals"] + 1)

df_clean = rescale
df_clean

,Instrument,Date,GICS Industry Group Name,Earnings Per Share - Actual Surprise,Earnings Per Share - Actual Surprise AbsVals,CBOE Crude Oil ETF Volatility Index,90-Day AA Financial Commercial Paper Interest Rate,Inflation Risk Premium,"University of Michigan: Consumer Sentiment, Index 1966:Q1=100",Unemployment Rate
0,AVY.N,2013-01-01,Materials,11.178,-0.163170,-1.005294,-0.788065,-0.374525,-0.744622,1.362950
1,AVY.N,2013-01-04,Materials,2.482,-0.490329,-0.945945,-0.810075,-0.361500,-0.334332,1.253195
2,AVY.N,2013-01-07,Materials,1.068,-0.555328,-0.905655,-0.828005,0.937254,-0.342537,1.088562
3,AVY.N,2013-01-10,Materials,8.095,-0.267337,-1.183895,-0.827825,0.661257,-0.722740,0.923930
4,AVY.N,2014-01-01,Materials,1.471,-0.536368,-1.195206,-0.827407,0.958285,-0.394508,0.777590
...,...,...,...,...,...,...,...,...,...,...
19196,POOL.OQ,2021-01-10,Retailing,17.194,0.013332,0.303321,-0.813038,-0.083766,-1.299883,-0.576057
19197,POOL.OQ,2022-01-01,Retailing,40.267,0.496103,0.920000,-0.490075,-0.105142,-1.855143,-0.795567
19198,POOL.OQ,2022-01-04,Retailing,34.342,0.392592,0.777108,0.426826,1.337373,-2.287316,-0.905322
19199,POOL.OQ,2022-01-07,Retailing,1.503,-0.534878,0.739057,1.902525,0.424584,-2.432286,-0.923614


In [59]:
df_panel = df_clean.copy()

#### Fixed Effects

In a fixed effects panel regression, the individual-specific effects are modeled as fixed variables that do not vary across time. This means that the coefficients of the independent variables are estimated based on the within-entity variation in the data, which eliminates the effect of time-invariant unobserved heterogeneity.

Fixed effects models are useful when there are time-invariant unobserved variables that may affect the dependent variable, but are not included in the model. By modeling the individual-specific effects as fixed variables, fixed effects models can control for this unobserved heterogeneity and estimate the coefficients of the independent variables based on the within-entity variation, which provides more efficient estimates of the coefficients.

One limitation of fixed effects models is that they do not allow for testing the effect of time-invariant variables on the dependent variable. In addition, fixed effects models may suffer from the incidental parameter problem, which may lead to biased estimates of the coefficients of the independent variables in the presence of a large number of fixed effects.

In [60]:
df_clean['Instrument'] = df_clean['Instrument'].astype('category')
df_clean['Date'] = pd.to_datetime(df_clean['Date'])

# set the index to be the time variable and the cross-sectional variable
df_clean.set_index(['Instrument', 'Date'], inplace=True)

Regression with absolute surprise values

In [61]:
y = df_clean.loc[:, "Earnings Per Share - Actual Surprise AbsVals"]
X = df_clean.loc[:, "CBOE Crude Oil ETF Volatility Index":]

# perform the fixed effects panel regression
fixed_effects_model = PanelOLS(y, X, entity_effects=True, time_effects=True, drop_absorbed=True)

# fit the model and print the summary statistics
fixed_effects_results = fixed_effects_model.fit()
fixed_effects_results.summary

Dep. Variable:,Earnings Per Share - Actual Surprise AbsVals,R-squared:,8.907e-08
Estimator:,PanelOLS,R-squared (Between):,-9.385e+18
No. Observations:,19201,R-squared (Within):,-2.279e+20
Date:,"Tue, Mar 07 2023",R-squared (Overall):,-1.391e+20
Time:,20:21:04,Log-likelihood,-8488.7
Cov. Estimator:,Unadjusted,,
,,F-statistic:,0.0003
Entities:,502,P-value,1.0000
Avg Obs:,38.249,Distribution:,"F(5,18655)"
Min Obs:,4.0000,,
Max Obs:,40.000,F-statistic (robust):,0.0003


By Industry: example tech industry

In [62]:
group = df_panel[df_panel['GICS Industry Group Name'] == 'Technology Hardware & Equipment']

group['Instrument'] = group['Instrument'].astype('category')
group['Date'] = pd.to_datetime(group['Date'])
group.set_index(['Instrument', 'Date'], inplace=True)

y = group.loc[:, "Earnings Per Share - Actual Surprise AbsVals"]
X = group.loc[:, "CBOE Crude Oil ETF Volatility Index":]

model = PanelOLS(y, X, entity_effects=True, time_effects=True).fit()
model

/var/folders/tj/07dch2wn75v90cdvrd8w0xm80000gn/T/ipykernel_74505/2986103428.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  group['Instrument'] = group['Instrument'].astype('category')
/var/folders/tj/07dch2wn75v90cdvrd8w0xm80000gn/T/ipykernel_74505/2986103428.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  group['Date'] = pd.to_datetime(group['Date'])


Dep. Variable:,Earnings Per Share - Actual Surprise AbsVals,R-squared:,0.0000
Estimator:,PanelOLS,R-squared (Between):,-0.0016
No. Observations:,731,R-squared (Within):,-2.2666
Date:,"Tue, Mar 07 2023",R-squared (Overall):,-1.1142
Time:,20:21:04,Log-likelihood,-30.601
Cov. Estimator:,Unadjusted,,
,,F-statistic:,0.0000
Entities:,19,P-value,1.0000
Avg Obs:,38.474,Distribution:,"F(5,668)"
Min Obs:,28.000,,
Max Obs:,40.000,F-statistic (robust):,2.333e-31


# Checking for COVID Years

In [63]:
df_clean = df_panel.copy()
df_clean

,Instrument,Date,GICS Industry Group Name,Earnings Per Share - Actual Surprise,Earnings Per Share - Actual Surprise AbsVals,CBOE Crude Oil ETF Volatility Index,90-Day AA Financial Commercial Paper Interest Rate,Inflation Risk Premium,"University of Michigan: Consumer Sentiment, Index 1966:Q1=100",Unemployment Rate
0,AVY.N,2013-01-01,Materials,11.178,-0.163170,-1.005294,-0.788065,-0.374525,-0.744622,1.362950
1,AVY.N,2013-01-04,Materials,2.482,-0.490329,-0.945945,-0.810075,-0.361500,-0.334332,1.253195
2,AVY.N,2013-01-07,Materials,1.068,-0.555328,-0.905655,-0.828005,0.937254,-0.342537,1.088562
3,AVY.N,2013-01-10,Materials,8.095,-0.267337,-1.183895,-0.827825,0.661257,-0.722740,0.923930
4,AVY.N,2014-01-01,Materials,1.471,-0.536368,-1.195206,-0.827407,0.958285,-0.394508,0.777590
...,...,...,...,...,...,...,...,...,...,...
19196,POOL.OQ,2021-01-10,Retailing,17.194,0.013332,0.303321,-0.813038,-0.083766,-1.299883,-0.576057
19197,POOL.OQ,2022-01-01,Retailing,40.267,0.496103,0.920000,-0.490075,-0.105142,-1.855143,-0.795567
19198,POOL.OQ,2022-01-04,Retailing,34.342,0.392592,0.777108,0.426826,1.337373,-2.287316,-0.905322
19199,POOL.OQ,2022-01-07,Retailing,1.503,-0.534878,0.739057,1.902525,0.424584,-2.432286,-0.923614


In [64]:
covid_start = pd.to_datetime("2020-01-01")

df_covid = df_clean[df_clean["Date"] >= covid_start]

df_nocovid = df_clean[df_clean["Date"] < covid_start]

In [65]:
df_covid['Instrument'] = df_covid['Instrument'].astype('category')
df_covid['Date'] = pd.to_datetime(df_covid['Date'])

# set the index to be the time variable and the cross-sectional variable
df_covid.set_index(['Instrument', 'Date'], inplace=True)

/var/folders/tj/07dch2wn75v90cdvrd8w0xm80000gn/T/ipykernel_74505/2704597447.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_covid['Instrument'] = df_covid['Instrument'].astype('category')
/var/folders/tj/07dch2wn75v90cdvrd8w0xm80000gn/T/ipykernel_74505/2704597447.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_covid['Date'] = pd.to_datetime(df_covid['Date'])


In [66]:
df_nocovid['Instrument'] = df_nocovid['Instrument'].astype('category')
df_nocovid['Date'] = pd.to_datetime(df_nocovid['Date'])

# set the index to be the time variable and the cross-sectional variable
df_nocovid.set_index(['Instrument', 'Date'], inplace=True)

/var/folders/tj/07dch2wn75v90cdvrd8w0xm80000gn/T/ipykernel_74505/1939219620.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_nocovid['Instrument'] = df_nocovid['Instrument'].astype('category')
/var/folders/tj/07dch2wn75v90cdvrd8w0xm80000gn/T/ipykernel_74505/1939219620.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_nocovid['Date'] = pd.to_datetime(df_nocovid['Date'])


# COVID

In [67]:
y = df_covid.loc[:, "Earnings Per Share - Actual Surprise AbsVals"]
X = df_covid.loc[:, "CBOE Crude Oil ETF Volatility Index":]

# perform the fixed effects panel regression
fixed_effects_model = PanelOLS(y, X, entity_effects=True, time_effects=True)

# fit the model and print the summary statistics
fixed_effects_results = fixed_effects_model.fit()
fixed_effects_results.summary


Dep. Variable:,Earnings Per Share - Actual Surprise AbsVals,R-squared:,6.414e-07
Estimator:,PanelOLS,R-squared (Between):,-1.456e+22
No. Observations:,5883,R-squared (Within):,-5.996e+22
Date:,"Tue, Mar 07 2023",R-squared (Overall):,-4.327e+22
Time:,20:21:05,Log-likelihood,-2851.1
Cov. Estimator:,Unadjusted,,
,,F-statistic:,0.0007
Entities:,501,P-value,1.0000
Avg Obs:,11.743,Distribution:,"F(5,5366)"
Min Obs:,4.0000,,
Max Obs:,12.000,F-statistic (robust):,0.0007


In [68]:
y = df_nocovid.loc[:, "Earnings Per Share - Actual Surprise AbsVals"]
X = df_nocovid.loc[:, "CBOE Crude Oil ETF Volatility Index":]

# perform the fixed effects panel regression
fixed_effects_model = PanelOLS(y, X, entity_effects=True, time_effects=True)

# fit the model and print the summary statistics
fixed_effects_results = fixed_effects_model.fit()
fixed_effects_results.summary


Dep. Variable:,Earnings Per Share - Actual Surprise AbsVals,R-squared:,1.629e-08
Estimator:,PanelOLS,R-squared (Between):,-9.049e+18
No. Observations:,13318,R-squared (Within):,-8.106e+19
Date:,"Tue, Mar 07 2023",R-squared (Overall):,-4.365e+19
Time:,20:21:05,Log-likelihood,-4506.7
Cov. Estimator:,Unadjusted,,
,,F-statistic:,4.165e-05
Entities:,498,P-value,1.0000
Avg Obs:,26.743,Distribution:,"F(5,12788)"
Min Obs:,1.0000,,
Max Obs:,28.000,F-statistic (robust):,4.165e-05


### Based on these results we can conclude that macro factors are a bad indicator for prediction